In [1]:
import numpy as np
import scipy.linalg as la
import pandas as pd

# Aula 01 - Utilização prática da Matriz Insumo Produto
Objetivos : 
* Principais conceitos da Matriz Insumo Produto
* 


# Extrair informações da MIP

In [2]:
#Produção total
V = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Recursos',skiprows=5,skipfooter=4,usecols=[*range(11,78)],header=None)
V = np.transpose(V)
#Impostos
Imp = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Recursos',skiprows=5,skipfooter=4,usecols=[8],header=None)
#Consumo intermediário produção nacional
Un = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Nacional',skiprows=5,skipfooter=4,usecols=[*range(2,69)],header=None)
#Demanda Final Nacional
Fn = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Nacional',skiprows=5,skipfooter=4,usecols=[*range(70,76)],header=None)

#Consumo intermediário produção importados
Um = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Importado',skiprows=5,skipfooter=4,usecols=[*range(2,69)],header=None)
#Demanda Final Nacional importados
Fm = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Importado',skiprows=5,skipfooter=4,usecols=[*range(70,76)],header=None)


# Extrair informações da TRU


In [3]:
#Dados da TRU
planilha = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2019.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA
VAB = planilha.iloc[0,:].to_numpy()#Valor adicionado bruto
W = planilha.loc[1,:].to_numpy()#Remuneracoes
RMB = planilha.loc[8,:].to_numpy()#Rendimento misto bruto
Out_Imp = planilha.loc[10,:].to_numpy()# outros impostos e subsidios
Sub = planilha.loc[11,:].to_numpy()# outros impostos e subsidios
Imp_ou_sub = Out_Imp + Sub
VBP = planilha.iloc[12,:].to_numpy()#Valor bruto de produção
o = planilha.loc[13,:]# ocupações

#Matriz para agregação de 68 atividades em 67 da TRU
agg_68_67 = np.concatenate((np.eye(67,41),np.concatenate((np.zeros((40,27)),np.eye(27,27)))),axis=1)
#Transformação nas matrizes para 67 atividades
VAB = np.matmul(agg_68_67,VAB.transpose())
W = np.matmul(agg_68_67,W.transpose())
RMB = np.matmul(agg_68_67,RMB.transpose())
Out_Imp = np.matmul(agg_68_67,Out_Imp.transpose())
Sub = np.matmul(agg_68_67,Sub.transpose())
Imp_ou_sub = np.matmul(agg_68_67,Imp_ou_sub.transpose())
VBP = np.matmul(agg_68_67,VBP.transpose())

o = np.matmul(agg_68_67,o.transpose())




# Derivar informações da MIP

In [4]:
V.shape

(67, 126)

In [5]:
g = np.matmul(V,np.ones(126)) ## Produção total por atividade No caso da MIP estimada, o transporte aquaviário foi fundido com o terrestre de cargas
q = np.transpose(np.matmul(np.transpose(np.ones(67)),V)) ## Produção total por produto
D = np.matmul(V,np.linalg.inv(np.diag(q))) # market share

Bn = np.matmul(Un,np.linalg.inv(np.diag(g))) # consumo intermediário nacional
A = np.matmul(D,Bn) # coeficientes técnicos intersetoriais nacionais

fn = np.matmul(Fn,np.transpose(np.ones(6))) # soma da demanda final
e = np.matmul(D,fn) #Demanda final ( nacional )

Bm = np.matmul(Um,np.linalg.inv(np.diag(g))) # consumo intermediário importado

A_plus = np.matmul(D,Bn+Bm) # coeficientes técnicos ampliada ( contém insumos nacionais e importados )
L = np.linalg.inv(np.eye(A.shape[0])-A) # leontief

l=np.matmul(np.linalg.inv(np.diag(g)),o)#Postos de trabalho por unidade (Milhão) de produção 



In [9]:
D.shape

(67, 126)

# Cálculo em um modelo fechado

In [7]:
sum(W)

3217680.0

In [8]:
cr =  W/VBP #Coeficientes de remuneração do trabalho ( renda )
ch = Fn.loc[:,Fn.columns[3]]/sum(W)

In [9]:
Fn.loc[:,Fn.columns[3]].shape

(126,)

# Cálculo dos multiplicadores

In [10]:
w = np.matmul(np.linalg.inv(np.diag(g)),W) # participação de remuneracoes pela produção total, para cada atividade
wa = np.matmul(np.linalg.inv(np.diag(g)),W+RMB) # participação de remuneracoes pela produção total, para cada atividade

y=np.matmul(np.linalg.inv(np.diag(g)),VAB) # participação de valor agregado pela produção total, para cada atividade
tp=np.matmul(np.matmul(np.linalg.inv(np.diag(g)),D),Imp)#Divisão de impostos pelo market share de cada produto

to=np.matmul(np.matmul(np.diag(y),np.linalg.inv(np.diag(VAB))),Imp_ou_sub)#Outros impostos sobre a produção

tn = tp[8].to_numpy()+to #Total de impostos sobre produção nacional


In [11]:

massa_salarial = np.matmul(w.transpose(),np.matmul(L,e))
pib = np.matmul(np.matmul(y.transpose()+tp.transpose(),L),e.to_numpy())
Valor_Agregado = np.matmul(np.matmul(y.transpose(),L),e)

In [12]:
#Calcular o incremento de uma unidade (milhão) na atividade identificadada por 1 no vetor arr
arr = np.zeros(67)
arr[7]=1

print("Aumento no consumo intermediário importado de {:0.2f} ".format(np.matmul(np.ones(67),np.matmul(np.matmul(np.matmul(D,Bm),L),arr)*1000000)))#aumento de consumo intermediário importado
print("Aumento no valor agregado de produção de {:0.2f} ".format(np.matmul(np.matmul(y.transpose(),L),arr)*1000000))
print("Aumento no PIB de {:0.2f} ".format(np.matmul(np.matmul(y.transpose()+tp.transpose(),L),arr).values[0]*1000000))
print("Aumento nas ocupações (postos de trabalho) de {:0.2f} ".format(np.matmul(np.matmul(l,L),arr)))
print("Aumento na massa salarial de {:0.2f} ".format(np.matmul(np.matmul(w,L),arr)*1000000))
print("Aumento na massa salarial ampliada de {:0.2f} ".format(np.matmul(np.matmul(wa,L),arr)*1000000))

Aumento no consumo intermediário importado de 120591.27 
Aumento no valor agregado de produção de 792550.11 
Aumento no PIB de 855268.03 
Aumento nas ocupações (postos de trabalho) de 22.74 
Aumento na massa salarial de 364504.47 
Aumento na massa salarial ampliada de 508903.16 


# Cálculo dos multiplicadores para trás - Total

In [13]:
mul_prod = np.matmul(np.ones(67).transpose(),L)#Multiplicador para produção
mul_sal = np.matmul(w.transpose(),L)#multiplicador de remunerações
mul_sal_ajus = np.matmul(wa.transpose(),L)#multiplicador de remunerações ajustado
mul_imp = np.matmul(tp[8].transpose(),L)#multiplicador de impostos
mul_ocup = np.matmul(l.transpose(),L)#Multiplicador de ocupações


In [14]:
poder_dispersao = 67*mul_prod / np.matmul(np.matmul(np.ones(67).transpose(),L),np.ones(67).transpose()) #multiplicador de produção normalizado

In [15]:
#Bn = np.matmul(np.linalg.inv(np.diag(g)),D) # consumo intermediário nacional
#Sensibilidade de dispersão (Gosh)


B = np.matmul(np.matmul(np.linalg.inv(np.diag(g)),D),Un) #coeficientes técnicos a jusante
G = np.linalg.inv(np.eye(B.shape[0])-B) #matriz inversa de Gosh
mul_prod_jus = np.matmul(G,np.ones(67))
sensibilidade_dispersao = 67*mul_prod_jus / np.matmul(np.matmul(np.ones(67),G),np.ones(67)) #multiplicador a jusante normalizado


# Modelo antigo de cálculo

In [115]:

lista_valores_estimados_e_atuais = ["30783.49	33800.59	26985.95","23827.59	27125.16	21237.93","15568.81	17353.37	14090.02","10274.08	12203.09	7914.03","9189.94	11750.29	8316.09","8766.51	22180.82	3835.61","7149.87	8764.71	6767.39"]
lista_setores_indices = []

lista_setores_indices.append(print_indice_setor("Fabricação de químicos orgânicos e inorgânicos, resinas e elastômeros"))
lista_setores_indices.append(print_indice_setor("Fabricação de automóveis, caminhões e ônibus, exceto peças"))
lista_setores_indices.append(print_indice_setor("Fabricação de máquinas e equipamentos mecânicos"))
lista_setores_indices.append(print_indice_setor("Fabricação de máquinas e equipamentos elétricos"))
lista_setores_indices.append(print_indice_setor("Fabricação de produtos farmoquímicos e farmacêuticos"))
lista_setores_indices.append(print_indice_setor("Fabricação de outros equipamentos de transporte, exceto veículos automotores"))
lista_setores_indices.append(print_indice_setor("Fabricação de equipamentos de informática, produtos eletrônicos e ópticos"))

df_setores = pd.DataFrame(data=lista_setores_indices,columns=['Atividades','Indice','Multiplicador de produção','Multiplicador de remuneração','Multiplicador de remuneração ajustado','Multiplicador de impostos','Multiplicador de ocupações'])

Setor Fabricação de químicos orgânicos e inorgânicos, resinas e elastômeros possui índice 20
Setor Fabricação de automóveis, caminhões e ônibus, exceto peças possui índice 32
Setor Fabricação de máquinas e equipamentos mecânicos possui índice 31
Setor Fabricação de máquinas e equipamentos elétricos possui índice 30
Setor Fabricação de produtos farmoquímicos e farmacêuticos possui índice 23
Setor Fabricação de outros equipamentos de transporte, exceto veículos automotores possui índice 34
Setor Fabricação de equipamentos de informática, produtos eletrônicos e ópticos possui índice 29


In [116]:
for i,j in df_setores.iterrows():
    df_setores.loc[i,"Estimativa de crescimento pessimista (US$)"] = lista_valores_estimados_e_atuais[i].split("\t")[0]
    df_setores.loc[i,"Estimativa crescimento otimista (US$)"] = lista_valores_estimados_e_atuais[i].split("\t")[1]
    df_setores.loc[i,"Valores atuais ( 2021 ) (US$)"] = lista_valores_estimados_e_atuais[i].split("\t")[2]
    

In [117]:
df_setores['Percentuais do setor'] = 1.
df_setores.loc[0,"Percentuais do setor"] = 0.53
df_setores.loc[1,"Percentuais do setor"] = 0.4

In [135]:
cambio = 5.25 # data 17/04/2024
colunas_estimativas = ["Estimativa de crescimento pessimista (US$)","Estimativa crescimento otimista (US$)","Valores atuais ( 2021 ) (US$)"]
multiplicadores = ['Multiplicador de produção','Multiplicador de remuneração','Multiplicador de remuneração ajustado','Multiplicador de impostos','Multiplicador de ocupações']
label = " Impacto {} na {}"
label_estimativa =""

for i in colunas_estimativas:
    df_setores[i] = df_setores[i].astype(float)
    df_setores[i[:-6]+" (R$) "] = df_setores[i]*df_setores['Percentuais do setor']*cambio
    if "pessimista" in i:
        label_estimativa="pessimista"
    elif "otimista" in i:
        label_estimativa="otimista"
    elif "atuais" in i:
        label_estimativa="atual"
    for mul in multiplicadores:
        df_setores[label.format(label_estimativa,mul[17:])] = df_setores[mul]*df_setores[i[:-6]+" (R$) "]


In [136]:
df_setores

,Atividades,Indice,Multiplicador de produção,Multiplicador de remuneração,Multiplicador de remuneração ajustado,Multiplicador de impostos,Multiplicador de ocupações,Estimativa de crescimento pessimista (US$),Estimativa crescimento otimista (US$),Valores atuais ( 2021 ) (US$),...,Impacto otimista na produção,Impacto otimista na remuneração,Impacto otimista na remuneração ajustado,Impacto otimista na impostos,Impacto otimista na ocupações,Impacto atual na produção,Impacto atual na remuneração,Impacto atual na remuneração ajustado,Impacto atual na impostos,Impacto atual na ocupações
0,Fabricação de químicos orgânicos e inorgânicos...,20,1.987735,0.225784,0.254268,0.110599,4.877845,30783.49,33800.59,26985.95,...,186946.755968,21235.030631,23913.918942,10401.864225,458762.056516,149255.850540,16953.771365,19092.560836,8304.712666,366269.639643
1,"Fabricação de automóveis, caminhões e ônibus, ...",32,2.277932,0.368776,0.405587,0.085764,8.012436,23827.59,27125.16,21237.93,...,129757.464625,21006.502128,23103.380346,4885.351069,456411.093248,101594.974949,16447.262310,18089.035219,3825.037125,357351.877357
2,Fabricação de máquinas e equipamentos mecânicos,31,1.888972,0.393443,0.424634,0.078421,8.434712,15568.81,17353.37,14090.02,...,172095.181706,35844.712612,38686.371999,7144.516089,768446.020038,139732.199114,29104.013664,31411.291017,5800.969759,623937.586259
3,Fabricação de máquinas e equipamentos elétricos,30,2.038429,0.368945,0.401322,0.086267,8.232016,10274.08,12203.09,7914.03,...,130594.446753,23636.939328,25711.214222,5526.790640,527394.188242,84693.988935,15329.186866,16674.409571,3584.271437,342029.226005
4,Fabricação de produtos farmoquímicos e farmacê...,23,1.682166,0.338133,0.376008,0.043913,6.242270,9189.94,11750.29,8316.09,...,103771.206044,20859.063722,23195.561303,2708.957883,385079.517813,73442.501323,14762.686813,16416.307631,1917.223963,272534.203606
5,Fabricação de outros equipamentos de transport...,34,1.717019,0.381328,0.404196,0.055722,6.583870,8766.51,22180.82,3835.61,...,199945.611341,44405.389935,47068.355310,6488.798833,766687.049821,34575.520036,7678.785441,8139.277732,1122.073111,132579.071250
6,"Fabricação de equipamentos de informática, pro...",29,1.656178,0.234018,0.260813,0.108588,5.395175,7149.87,8764.71,6767.39,...,76208.581578,10768.272290,12001.240621,4996.632899,248257.492789,58842.014498,8314.376427,9266.373419,3857.989998,191684.068740


In [137]:
df_setores.to_excel('./dados/estimativa_NIB.xlsx')

# Teste de derivação da MIP a partir da TRU ( Andamento )

In [12]:
ConsumoIntermediario = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2015.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA
Demanda = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2015.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA


In [6]:
Q = np.array([[280,120],[12,8]])
q = np.array([575,20])

print(Q)
print(q)

[[280 120]
 [ 12   8]]
[575  20]


In [14]:
A= np.matmul(Q,np.linalg.inv(np.diag(q)))

In [19]:
eigvector,eigvalues = la.eig(np.matrix.transpose(A))

In [38]:
lambdas = max(eigvector).real
pos_vec = np.argmax(eigvector)

In [39]:
max_eigvectors = eigvalues[:,pos_vec].reshape(2,1)

In [40]:
r = (1-lambdas)/lambdas

In [37]:
lambdas.real

0.8

In [45]:
max_eigvectors/max_eigvectors[0]

array([[ 1.],
       [15.]])

## Importação de dados reais

In [1]:
import pandas as pd


Undefined variable: import
